# ATTITUDE_CONVERT

作成：電シス 浅井2021.8.30

参考サイト：https://qiita.com/momomo_rimoto/items/3a245736c5fd90fe8270

In [3]:
import numpy as np
import quaternion

import sys
sys.path.append('../')
sys.path.append('../sensor')

In [4]:
#座標系クラスのインポート
from COORDINATION_SYSTEMS import ECEF, LLA, NED, FRD, AC_ATTITUDE, LOCAL, SENSOR_ATTITUDE, FOCAL_POSITION, FOCAL_ANGLE

In [5]:
#WGS84座標変換クラスのインポート
from WGS84_COORDINATE_CONVERT import WGS84_COORDINATION_CONVERT

## クラス定義

In [6]:
class ATTITUDE_CONVERT:
    def ned2frd(self, ned = NED(), ac_attitude = AC_ATTITUDE()):
        #入力：　北、東、南 ：単位m
        #入力：ロール、ピッチ、ヨ― ：単位°
        #出力： 前、右、下 単位m
        #オイラーシーケンス：ヨー→ピッチ→ロール
        self.ned = ned
        
        self.ac_attitude = ac_attitude        
        q1=quaternion.from_rotation_vector([0,0,np.radians(self.ac_attitude.yaw)])
        q2=quaternion.from_rotation_vector([0,np.radians(self.ac_attitude.pitch),0])
        q3=quaternion.from_rotation_vector([np.radians(self.ac_attitude.roll),0,0])

        quat=q3.inverse()*q2.inverse()*q1.inverse()
        rot = quaternion.as_rotation_matrix(quat)
        
        self.frd = FRD()
        self.frd.fwd, self.frd.rh, self.frd.dwn = np.matmul(rot, np.array([self.ned.north, self.ned.east, self.ned.down]))
              
        return self.frd
    
    def frd2ned(self, frd = FRD(), ac_attitude = AC_ATTITUDE()):
        #入力：　前、右、下 単位m
        #入力：ロール、ピッチ、ヨ― 単位°
        #出力： 北、東、南 ：単位m
        #オイラーシーケンス：ヨー→ピッチ→ロール
        
        self.frd = frd
               
        self.ac_attitude = ac_attitude
        q1=quaternion.from_rotation_vector([0,0,np.radians(self.ac_attitude.yaw)])
        q2=quaternion.from_rotation_vector([0,np.radians(self.ac_attitude.pitch),0])
        q3=quaternion.from_rotation_vector([np.radians(self.ac_attitude.roll),0,0])
        
        quat=q1*q2*q3
        rot = quaternion.as_rotation_matrix(quat)
        
        self.ned = NED()
        self.ned.north, self.ned.east, self.ned.down = np.matmul(rot, np.array([self.frd.fwd, self.frd.rh, self.frd.dwn]))
        
        return self.ned
    
    def local2ned(self, local= LOCAL(), lla = LLA() ):
        #入力：　局所X、局所Y、局所Z 単位m
        #入力：緯度、経度　単位°
        #出力： 北、東、南 ：単位m
        #オイラーシーケンス：経度→緯度
        
        self.local = local
        self.lla = lla
        
        q1=quaternion.from_rotation_vector([0,np.radians(-90),0])
        q2=quaternion.from_rotation_vector([np.radians(self.lla.lon),0,0])
        q3=quaternion.from_rotation_vector([0,np.radians(-self.lla.lat),0])
        
        quat = q3.inverse()*q2.inverse()*q1.inverse()
        
        rot = quaternion.as_rotation_matrix(quat)
        
        self.ned = NED()
        self.ned.north, self.ned.east, self.ned.down = np.matmul(rot, np.array([self.local.local_x, self.local.local_y, self.local.local_z]))
        
        return self.ned
    
    def ned2local(self, ned = NED(), lla = LLA()):
        #入力：北、東、南 ：単位m　
        #入力：緯度、経度　単位°
        #出力： 局所X、局所Y、局所Z 単位m
        #オイラーシーケンス：経度→緯度
        
        self.ned = ned
        self.lla = lla

        q1=quaternion.from_rotation_vector([0,np.radians(-90),0])
        q2=quaternion.from_rotation_vector([np.radians(self.lla.lon),0,0])
        q3=quaternion.from_rotation_vector([0,np.radians(-self.lla.lat),0])
        
        quat=q1*q2*q3
        
        rot = quaternion.as_rotation_matrix(quat)
        
        self.local = LOCAL()
        self.local.local_x, self.local.local_y, self.local.local_z = np.matmul(rot, np.array([self.ned.north, self.ned.east, self.ned.down]))
        
        return self.local
    
    def frd2focal(self, frd = FRD(), sensor_attitude =SENSOR_ATTITUDE() ):
        #入力：前、右、下 ：単位m　
        #入力：アジマス、エレベーション、スピン　：単位°
        #出力： 水平右、垂直下、奥行前 ：単位m
        #オイラーシーケンス：アジマス→エレベーション→スピン
        self.frd = frd
        self.sensor_attitude = sensor_attitude
        
        q1=quaternion.from_rotation_vector([0,0,np.radians(self.sensor_attitude.az)])
        q2=quaternion.from_rotation_vector([0,np.radians(self.sensor_attitude.el),0])
        q3=quaternion.from_rotation_vector([np.radians(self.sensor_attitude.spin),0,0])
        q4=quaternion.from_rotation_vector([0,0,np.radians(90)])
        q5=quaternion.from_rotation_vector([np.radians(90),0,0])

        quat=q5.inverse()*q4.inverse()*q3.inverse()*q2.inverse()*q1.inverse()
        rot = quaternion.as_rotation_matrix(quat)
        
        self.focal_position = FOCAL_POSITION()
        self.focal_position.horizontal, self.focal_position.vertical, self.focal_position.depth = np.matmul(rot, np.array([self.frd.fwd, self.frd.rh, self.frd.dwn]))
        
        return self.focal_position

    def focal2frd(self, focal_position = FOCAL_POSITION(), sensor_attitude =SENSOR_ATTITUDE()):
        #入力： 水平右、垂直下、奥行前 ：単位m
        #入力：アジマス、エレベーション、スピン　：単位°
        #出力：前、右、下 ：単位m　
        #オイラーシーケンス：アジマス→エレベーション→スピン
        self.focal_position = focal_position
        self.sensor_attitude = sensor_attitude
        
        q1=quaternion.from_rotation_vector([0,0,np.radians(self.sensor_attitude.az)])
        q2=quaternion.from_rotation_vector([0,np.radians(self.sensor_attitude.el),0])
        q3=quaternion.from_rotation_vector([np.radians(self.sensor_attitude.spin),0,0])
        q4=quaternion.from_rotation_vector([0,0,np.radians(90)])
        q5=quaternion.from_rotation_vector([np.radians(90),0,0])

        quat=q1*q2*q3*q4*q5
        rot = quaternion.as_rotation_matrix(quat)
        
        self.frd = FRD()
        self.frd.fwd, self.frd.rh, self.frd.dwn = np.matmul(rot, np.array([self.focal_position.horizontal, self.focal_position.vertical, self.focal_position.depth]))
                
        return self.frd
    

## 使い方

機体位置を原点に、NED座標で表された位置(真北、東、真下）を、機体固定座標(前方、右方、下方）で表すサンプル<br>角度単位は°

In [7]:
ned = NED(north =0, east =1, down =0)
ac_attitude = AC_ATTITUDE(roll = 0, pitch = -45, yaw =90)
frd = ATTITUDE_CONVERT().ned2frd(ned, ac_attitude)
print(frd.fwd, frd.rh, frd.dwn)

0.7071067811865474 3.3306690738754696e-16 -0.7071067811865474


機体位置を原点に、機体固定座標で表された位置（前方、右方、下方）を、NED座標（真北、東、真下）で表すサンプル<br>（角度単位は°）

In [8]:
frd = FRD(fwd =0.7071067811865474, rh =3.3306690738754696e-16, dwn =-0.7071067811865474)
ac_attitude = AC_ATTITUDE(roll = 0, pitch = -45, yaw =90)
ned = ATTITUDE_CONVERT().frd2ned(frd, ac_attitude)
print(ned.north, ned.east, ned.down)

-3.3306690738754696e-16 0.9999999999999998 5.551115123125783e-17


機体位置を原点に機体局所座標で表された位置（局所X、局所Y、局所Z）を、NED座標（真北、東、真下）で表すサンプル。<br>（角度単位は°）（局所X方向は、地球中心から北緯0°東経0°の地点を見た方向と平行）

In [9]:
local = LOCAL(local_x=0.2, local_y=0.4, local_z=0.6)
lla = LLA(lat=30,lon=90, alt=0)
ned = ATTITUDE_CONVERT().local2ned(local, lla)
print(ned.north, ned.east, ned.down)

0.31961524227066307 -0.19999999999999987 -0.6464101615137753


機体位置を原点に、NED座標で表された位置（真北、東、真下）を、機体局所座標で表された位置（局所X、局所Y、局所Z）で表すサンプル。<br>（角度単位は°）（局所X方向は、地球中心から北緯０°東経０°の地点を見た方向と平行）

In [10]:
ned = NED(north =0.31961524227066307, east= -0.19999999999999987, down =-0.6464101615137753)
lla = LLA(lat=30, lon=90, alt=0)
local = ATTITUDE_CONVERT().ned2local(ned, lla)
print(local.local_x, local.local_y, local.local_z)

0.19999999999999982 0.40000000000000013 0.6000000000000005


機体位置を原点に、機体固定座標で表された位置（前方、右方、下方）を、機体局所座標で表されたセンサ固定座標（水平、垂直、奥行）で表すサンプル。<br>（角度単位は°）（水平は右が正、垂直は下が正）

In [11]:
frd = FRD(fwd =0.7071067811865474, rh =3.3306690738754696e-16, dwn =-0.7071067811865474)
sensor_attitude =SENSOR_ATTITUDE(spin=0, el=90, az=0)
focal_position = ATTITUDE_CONVERT().frd2focal(frd, sensor_attitude)
print(focal_position.horizontal, focal_position.vertical, focal_position.depth)

6.470853991243017e-16 0.707106781186547 0.7071067811865467


機体位置を原点に、センサ固定座標で表された位置（水平、垂直、奥行）を、機体局所座標（前方、右下、下方）で表すサンプル。<br>（角度単位は°）（水平は右が正、垂直は下が正

In [12]:
focal_position = FOCAL_POSITION(horizontal= 6.470853991243017e-16, vertical=0.707106781186547, depth=0.7071067811865467)
sensor_attitude =SENSOR_ATTITUDE(spin=0, el=90, az=0)
frd = ATTITUDE_CONVERT().focal2frd(focal_position,sensor_attitude)
print(frd.fwd, frd.rh, frd.dwn)

0.7071067811865474 3.3306690738754726e-16 -0.7071067811865474
